In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
import panel as pn
import param
import requests
from langchain.agents import AgentType, initialize_agent, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents import load_tools
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.agents import tool
from langchain_openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
import os
import datetime
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.document_loaders import TextLoader

In [127]:
# create our vector embeddings 

embeddings = OpenAIEmbeddings()

# account for deprecation of LLM model
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

raw_documents = TextLoader("../data/data.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

docs = text_splitter.split_documents(documents)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)


llm = ChatOpenAI(temperature=0.0, model=llm_model, max_tokens=2048)
db = DocArrayInMemorySearch.from_documents(
    all_splits,
    embeddings
)
retriever = db.as_retriever()
# qa_stuff = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     verbose=True
# )
from langchain.prompts import PromptTemplate

prompt_template = """If the context is not relevant, 
        please answer the question by using your own knowledge about the topic
        
        {context}
        
        Question: {question}
        """

PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "question"]
        )

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                 chain_type="stuff", 
                                 retriever=retriever, 
                                 chain_type_kwargs=chain_type_kwargs,
                                verbose=True)


# from langchain.tools.retriever import create_retriever_tool

# retriever_tool = create_retriever_tool(
#     retriever,
#     "seminars_search",
#     "Search for information about questions related to the seminars. For any questions about seminars, you must use this tool!",
# )

In [128]:
@tool
def find_seminar_quote(text: str) -> str:
    """ Receives a question directly from the user about a seminar and returns a description of it from the transcript. To use this be aware that you should provide a good context
    for the question. Always input all the words from the user;
    """

    query = f"""Please gather more information about '''{text}''' and always send me a string, also an abstract of the topic"""
    response = qa_stuff.run(query)
    print("find_seminar_quote", response, flush=True)

    return response

In [129]:
find_seminar_quote("What is IOT?")



> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote IOT, or the Internet of Things, is a system of interrelated computing devices, mechanical and digital machines, objects, animals, or people that are provided with unique identifiers and the ability to transfer data over a network without requiring human-to-human or human-to-computer interaction. It is a concept that describes the connection of everyday objects to the internet and to one another, allowing them to send and receive data. This connectivity enables these objects to be remotely monitored and controlled, leading to increased efficiency, accuracy, and economic benefit. The evolution of IOT is leading to a more interconnected world where devices, systems, and people can communicate seamlessly, creating new opportunities for innovation and automation.


'IOT, or the Internet of Things, is a system of interrelated computing devices, mechanical and digital machines, objects, animals, or people that are provided with unique identifiers and the ability to transfer data over a network without requiring human-to-human or human-to-computer interaction. It is a concept that describes the connection of everyday objects to the internet and to one another, allowing them to send and receive data. This connectivity enables these objects to be remotely monitored and controlled, leading to increased efficiency, accuracy, and economic benefit. The evolution of IOT is leading to a more interconnected world where devices, systems, and people can communicate seamlessly, creating new opportunities for innovation and automation.'

In [131]:
find_seminar_quote("How much data is needed to transfer an hologram on our networks?")



> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't know.


"I don't know."

In [132]:
text = "How much data is needed to transfer an hologram on our networks?"
query = f"""Please gather more information about '''{text}''' and always send me a string, also an abstract of the topic"""
response = qa_stuff.run(query)
print("find_seminar_quote", response, flush=True)



> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't know.


In [142]:

def create_agent():
    prefix = """Your name is Chris. You are a specialist agent in Technology. You should have a friendly conversation with a human, 
    answering their questions as best as possible based on the seminars you have available. You have access to the following tools: """
    FORMAT_INSTRUCTIONS = """To use a tool, follow this format:
        Thought: Do I really need to use this tool? Yes
        Action: The action to take, should be one of [{tool_names}]
        Action Input: the exact user question
        Observation: The result of the action tool
        """
    suffix = """
    hystory: {chat_history}
    Question: {input}
    {agent_scratchpad}"""
    tools = [find_seminar_quote]

    prompt = ZeroShotAgent.create_prompt(
        tools,
        prefix=prefix,
        suffix=suffix,
        # format_instructions=FORMAT_INSTRUCTIONS,
        input_variables=[
            "inputs", "chat_history", "agent_scratchpad"],
    )
    memory = ConversationBufferWindowMemory(k=5, memory_key="chat_history")
    # memory = ConversationBufferMemory(memory_key="chat_history")
    print(prompt, flush=True)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    
    agent = ZeroShotAgent(
        llm_chain=llm_chain, tools=tools, verbose=True, max_iterations=3,)
    
    agent = AgentExecutor.from_agent_and_tools(
        agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True
    )
    print('criei o agente corretamente')
    return agent

In [143]:
agent = create_agent()

input_variables=['agent_scratchpad', 'chat_history', 'input'] template='Your name is Chris. You are a specialist agent in Technology. You should have a friendly conversation with a human, \n    answering their questions as best as possible based on the seminars you have available. You have access to the following tools: \n\nfind_seminar_quote(text: str) -> str - Receives a question directly from the user about a seminar and returns a description of it from the transcript. To use this be aware that you should provide a good context\n   for the question. Always input all the words from the user;\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [find_seminar_quote]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: th

In [135]:
find_seminar_quote("how much data is enough to pass an hologram through the net? ")



> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't know.


"I don't know."

In [136]:
text = "how much data is enough to pass an hologram through the net?"

querry = f"""Please gather more information about '''{text}''' and always send me a string"""

qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know."

In [137]:
agent.invoke(    {"input": "how much data is enough to pass an hologram through the net? "})



> Entering new AgentExecutor chain...
Thought: The user is asking about the amount of data needed to transmit a hologram over the internet.
Action: find_seminar_quote
Action Input: "data hologram pass through net"

> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't have information on "data hologram pass through net."

Observation: I don't have information on "data hologram pass through net."
Thought:I need to try a different approach to find information on transmitting holograms over the internet.
Action: find_seminar_quote
Action Input: "transmit hologram internet"

> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't have enough information to provide a detailed answer on transmitting holograms over the internet.

Observation: I don't have enough information to provide a detailed answer on transmitting holograms over the internet.
Thought:I need to try a more general approach to find information on hologram technology.
Ac

{'input': 'how much data is enough to pass an hologram through the net? ',
 'chat_history': '',
 'output': 'The amount of data needed to pass a hologram through the internet can vary depending on the size and complexity of the hologram. Generally, a large amount of data is required to transmit a high-quality hologram over the internet.'}

In [138]:
agent.invoke(    {"input": "Hi! "})



> Entering new AgentExecutor chain...
Thought: The user seems to be starting a conversation
Action: None
Final Answer: Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi! ',
 'chat_history': 'Human: how much data is enough to pass an hologram through the net? \nAI: The amount of data needed to pass a hologram through the internet can vary depending on the size and complexity of the hologram. Generally, a large amount of data is required to transmit a high-quality hologram over the internet.',
 'output': 'Hello! How can I assist you today?'}

In [139]:
agent.invoke(    {"input": "what do you know about medical diagnosis baesed on machine learning techniques"})



> Entering new AgentExecutor chain...
Thought: The user is asking about medical diagnosis based on machine learning techniques, I should use the find_seminar_quote tool to provide a relevant description from the seminar transcripts.
Action: find_seminar_quote
Action Input: "medical diagnosis based on machine learning techniques"

> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't have specific information on medical diagnosis based on machine learning techniques.

Observation: I don't have specific information on medical diagnosis based on machine learning techniques.
Thought:The tool didn't return any specific information on medical diagnosis based on machine learning techniques. I should try a different approach.
Action: find_seminar_quote
Action Input: "machine learning in healthcare"

> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't have specific information on machine learning in healthcare from the provided context

{'input': 'what do you know about medical diagnosis baesed on machine learning techniques',
 'chat_history': 'Human: how much data is enough to pass an hologram through the net? \nAI: The amount of data needed to pass a hologram through the internet can vary depending on the size and complexity of the hologram. Generally, a large amount of data is required to transmit a high-quality hologram over the internet.\nHuman: Hi! \nAI: Hello! How can I assist you today?',
 'output': "I don't have specific information on medical diagnosis based on machine learning techniques from the seminars available."}

In [144]:
agent.invoke({"input":"o Edge to Cloud continue and this notion of edge to help continue has to do it computation"})



> Entering new AgentExecutor chain...
Thought: The user is asking about the relationship between Edge to Cloud and computation.
Action: find_seminar_quote
Action Input: "Edge to Cloud computation"

> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't have specific information on "Edge to Cloud computation" from the provided context.

Observation: I don't have specific information on "Edge to Cloud computation" from the provided context.
Thought:I need to provide more context for the tool to find the relevant seminar quote.
Action: find_seminar_quote
Action Input: "Edge to Cloud"

> Entering new RetrievalQA chain...

> Finished chain.
find_seminar_quote I don't have the specific information on "Edge to Cloud" from the context provided.

Observation: I don't have the specific information on "Edge to Cloud" from the context provided.
Thought:I need to provide a more general question related to technology seminars.
Action: find_seminar_quote
Action Input: "te

{'input': 'o Edge to Cloud continue and this notion of edge to help continue has to do it computation',
 'chat_history': '',
 'output': "I'm sorry, but I don't have specific information on the relationship between Edge to Cloud and computation from the seminars available. Can you provide more details or ask about a different topic?"}

In [146]:
text = " What does the term Edge Computing means? "

querry = f"""Please gather more information about '''{text}''' and always send me a string"""

qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know."